In [3]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

In [4]:
#set parent folder as working directory
#os.chdir('..')
os.getcwd()

'c:\\Users\\isaldiviagonzatti\\OneDrive - Kiel Institut für Weltwirtschaft\\Documents\\General\\Ignacio'

In [433]:
dataF = r'raw_py'

#### Country names dict

In [177]:
country_to_abbrev = {
    "Andorra": "AD",
    "United Arab Emirates": "AE",
    "Afghanistan": "AF",
    "Antigua and Barbuda": "AG",
    "Anguilla": "AI",
    "Albania": "AL",
    "Armenia": "AM",
    "Angola": "AO",
    "Antarctica": "AQ",
    "Argentina": "AR",
    "American Samoa": "AS",
    "Austria": "AT",
    "Australia": "AU",
    "Aruba": "AW",
    "Åland Islands": "AX",
    "Azerbaijan": "AZ",
    "Bosnia and Herzegovina": "BA",
    "Barbados": "BB",
    "Bangladesh": "BD",
    "Belgium": "BE",
    "Burkina Faso": "BF",
    "Bulgaria": "BG",
    "Bahrain": "BH",
    "Burundi": "BI",
    "Benin": "BJ",
    "Saint Barthélemy": "BL",
    "Bermuda": "BM",
    "Brunei Darussalam": "BN",
    "Bolivia (Plurinational State of)": "BO",
    "Bonaire, Sint Eustatius and Saba": "BQ",
    "Brazil": "BR",
    "Bahamas": "BS",
    "Bhutan": "BT",
    "Bouvet Island": "BV",
    "Botswana": "BW",
    "Belarus": "BY",
    "Belize": "BZ",
    "Canada": "CA",
    "Cocos (Keeling) Islands": "CC",
    "Congo, Democratic Republic of the": "CD",
    "Central African Republic": "CF",
    "Congo": "CG",
    "Switzerland": "CH",
    "Côte d'Ivoire": "CI",
    "Cook Islands": "CK",
    "Chile": "CL",
    "Cameroon": "CM",
    "China": "CN",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Cuba": "CU",
    "Cabo Verde": "CV",
    "Curaçao": "CW",
    "Christmas Island": "CX",
    "Cyprus": "CY",
    "Czechia": "CZ",
    "Germany": "DE",
    "Djibouti": "DJ",
    "Denmark": "DK",
    "Dominica": "DM",
    "Dominican Republic": "DO",
    "Algeria": "DZ",
    "Ecuador": "EC",
    "Estonia": "EE",
    "Egypt": "EG",
    "Western Sahara": "EH",
    "Eritrea": "ER",
    "Spain": "ES",
    "Ethiopia": "ET",
    "Finland": "FI",
    "Fiji": "FJ",
    "Falkland Islands (Malvinas)": "FK",
    "Micronesia (Federated States of)": "FM",
    "Faroe Islands": "FO",
    "France": "FR",
    "Gabon": "GA",
    "United Kingdom of Great Britain and Northern Ireland": "UK", #original is GB, Eurostat uses UK
    "Grenada": "GD",
    "Georgia": "GE",
    "French Guiana": "GF",
    "Guernsey": "GG",
    "Ghana": "GH",
    "Gibraltar": "GI",
    "Greenland": "GL",
    "Gambia": "GM",
    "Guinea": "GN",
    "Guadeloupe": "GP",
    "Equatorial Guinea": "GQ",
    "Greece": "EL", #original ir GR, Eurostat uses EL
    "South Georgia and the South Sandwich Islands": "GS",
    "Guatemala": "GT",
    "Guam": "GU",
    "Guinea-Bissau": "GW",
    "Guyana": "GY",
    "Hong Kong": "HK",
    "Heard Island and McDonald Islands": "HM",
    "Honduras": "HN",
    "Croatia": "HR",
    "Haiti": "HT",
    "Hungary": "HU",
    "Indonesia": "ID",
    "Ireland": "IE",
    "Israel": "IL",
    "Isle of Man": "IM",
    "India": "IN",
    "British Indian Ocean Territory": "IO",
    "Iraq": "IQ",
    "Iran (Islamic Republic of)": "IR",
    "Iceland": "IS",
    "Italy": "IT",
    "Jersey": "JE",
    "Jamaica": "JM",
    "Jordan": "JO",
    "Japan": "JP",
    "Kenya": "KE",
    "Kyrgyzstan": "KG",
    "Cambodia": "KH",
    "Kiribati": "KI",
    "Comoros": "KM",
    "Saint Kitts and Nevis": "KN",
    "Korea (Democratic People's Republic of)": "KP",
    "Korea, Republic of": "KR",
    "Kuwait": "KW",
    "Cayman Islands": "KY",
    "Kazakhstan": "KZ",
    "Lao People's Democratic Republic": "LA",
    "Lebanon": "LB",
    "Saint Lucia": "LC",
    "Liechtenstein": "LI",
    "Sri Lanka": "LK",
    "Liberia": "LR",
    "Lesotho": "LS",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Latvia": "LV",
    "Libya": "LY",
    "Morocco": "MA",
    "Monaco": "MC",
    "Moldova, Republic of": "MD",
    "Montenegro": "ME",
    "Saint Martin (French part)": "MF",
    "Madagascar": "MG",
    "Marshall Islands": "MH",
    "North Macedonia": "MK",
    "Mali": "ML",
    "Myanmar": "MM",
    "Mongolia": "MN",
    "Macao": "MO",
    "Northern Mariana Islands": "MP",
    "Martinique": "MQ",
    "Mauritania": "MR",
    "Montserrat": "MS",
    "Malta": "MT",
    "Mauritius": "MU",
    "Maldives": "MV",
    "Malawi": "MW",
    "Mexico": "MX",
    "Malaysia": "MY",
    "Mozambique": "MZ",
    "Namibia": "NA",
    "New Caledonia": "NC",
    "Niger": "NE",
    "Norfolk Island": "NF",
    "Nigeria": "NG",
    "Nicaragua": "NI",
    "Netherlands": "NL",
    "Norway": "NO",
    "Nepal": "NP",
    "Nauru": "NR",
    "Niue": "NU",
    "New Zealand": "NZ",
    "Oman": "OM",
    "Panama": "PA",
    "Peru": "PE",
    "French Polynesia": "PF",
    "Papua New Guinea": "PG",
    "Philippines": "PH",
    "Pakistan": "PK",
    "Poland": "PL",
    "Saint Pierre and Miquelon": "PM",
    "Pitcairn": "PN",
    "Puerto Rico": "PR",
    "Palestine, State of": "PS",
    "Portugal": "PT",
    "Palau": "PW",
    "Paraguay": "PY",
    "Qatar": "QA",
    "Réunion": "RE",
    "Romania": "RO",
    "Serbia": "RS",
    "Russian Federation": "RU",
    "Rwanda": "RW",
    "Saudi Arabia": "SA",
    "Solomon Islands": "SB",
    "Seychelles": "SC",
    "Sudan": "SD",
    "Sweden": "SE",
    "Singapore": "SG",
    "Saint Helena, Ascension and Tristan da Cunha": "SH",
    "Slovenia": "SI",
    "Svalbard and Jan Mayen": "SJ",
    "Slovakia": "SK",
    "Sierra Leone": "SL",
    "San Marino": "SM",
    "Senegal": "SN",
    "Somalia": "SO",
    "Suriname": "SR",
    "South Sudan": "SS",
    "Sao Tome and Principe": "ST",
    "El Salvador": "SV",
    "Sint Maarten (Dutch part)": "SX",
    "Syrian Arab Republic": "SY",
    "Eswatini": "SZ",
    "Turks and Caicos Islands": "TC",
    "Chad": "TD",
    "French Southern Territories": "TF",
    "Togo": "TG",
    "Thailand": "TH",
    "Tajikistan": "TJ",
    "Tokelau": "TK",
    "Timor-Leste": "TL",
    "Turkmenistan": "TM",
    "Tunisia": "TN",
    "Tonga": "TO",
    "Turkey": "TR",
    "Trinidad and Tobago": "TT",
    "Tuvalu": "TV",
    "Taiwan, Province of China": "TW",
    "Tanzania, United Republic of": "TZ",
    "Ukraine": "UA",
    "Uganda": "UG",
    "United States Minor Outlying Islands": "UM",
    "United States of America": "US",
    "Uruguay": "UY",
    "Uzbekistan": "UZ",
    "Holy See": "VA",
    "Saint Vincent and the Grenadines": "VC",
    "Venezuela (Bolivarian Republic of)": "VE",
    "Virgin Islands (British)": "VG",
    "Virgin Islands (U.S.)": "VI",
    "Viet Nam": "VN",
    "Vanuatu": "VU",
    "Wallis and Futuna": "WF",
    "Samoa": "WS",
    "Yemen": "YE",
    "Mayotte": "YT",
    "South Africa": "ZA",
    "Zambia": "ZM",
    "Zimbabwe": "ZW",
}
    
# invert the dictionary
abbrev_to_country = dict(map(reversed, country_to_abbrev.items()))

In [196]:
countries=['Belgium','Bulgaria','Cyprus', 'Greece','Germany','Croatia','Italy', 
           'Denmark','Estonia','Spain','Finland','France','Ireland','Lithuania',
           'Latvia','Malta','Netherlands','Poland','Portugal', 'Romania',
           'Sweden','United Kingdom of Great Britain and Northern Ireland']


In [179]:
countryAbb = [x if x not in country_to_abbrev else country_to_abbrev[x] for x in countries]

### SDG Official Data

In [333]:
sdg14 = pd.read_excel(os.path.join(dataF, "Goal14_raw.xlsx"), sheet_name=1)

In [334]:
# filter columns 
sdg14 = sdg14[['Target', 'Indicator','SeriesCode', 'GeoAreaName',
                   'TimePeriod', 'Value','Units','SeriesDescription', ]]

In [335]:
# get name used for UK
sdg14.GeoAreaName[sdg14['GeoAreaName'].str.contains("United")].unique()

array(['United Kingdom of Great Britain and Northern Ireland'],
      dtype=object)

In [336]:
# filter countries
sdg14 = sdg14[sdg14['GeoAreaName'].isin(countries)]
# show indicators
pd.DataFrame(sdg14.groupby(['Indicator', 'SeriesCode', 'SeriesDescription', 'Units']).size())

0
Indicator SeriesCode        SeriesDescription                                  Units       
14.1.1    EN_MAR_BEALITSQ   Beach litter per square kilometer (Number)         NUMBER    90
          EN_MAR_BEALIT_BP  Beach litter originating from national land-bas... PERCENT  110
          EN_MAR_BEALIT_BV  Beach litter originating from national land-bas... TONNES   110
          EN_MAR_BEALIT_EXP Exported beach litter originating from national... TONNES   110
          EN_MAR_BEALIT_OP  Beach litter originating from national land-bas... PERCENT  110
          EN_MAR_BEALIT_OV  Beach litter originating from national land-bas... TONNES   110
          EN_MAR_CHLANM     Chlorophyll-a anomaly, remote sensing (%)          PERCENT  147
          EN_MAR_CHLDEV     Chlorophyll-a deviations, remote sensing (%)       PERCENT  372
14.2.1    EN_SCP_ECSYBA     Number of countries using ecosystem-based appro... NUMBER    10
14.3.1    ER_OAW_MNACD      Average marine acidity (pH) measured at agreed ... PH       434
14.4.1    ER_H2O_FWTL       Proportion of fish stocks within biologically s... PERCENT   23
14.5.1    ER_MRN_MARIN      Coverage of protected areas in relation to mari... PERCENT   22
          ER_MRN_MARINT     Protected marine area (Exclusive Economic Zones... KMSQ      22
          ER_MRN_MPA        Average proportion of Marine Key Biodiversity A... PERCENT  484
14.6.1    ER_REG_UNFCIM     Progress by countries in the degree of implemen... NUMBER    66
14.7.1    EN_SCP_FSHGDP     Sustainable fisheries as a proportion of GDP       PERCENT  440
14.a.1    ER_RDE_OSEX       National ocean science expenditure as a share o... PERCENT   48
14.b.1    ER_REG_SSFRAR     Degree of application of a legal/regulatory/pol... NUMBER    66
14.c.1    ER_UNCLOS_IMPLE   Score for the implementation of UNCLOS and its ... PERCENT   11
          ER_UNCLOS_RATACC  Score for the ratification of and accession to ... PERCENT   11

In [405]:
sdg14EZZ = sdg14[sdg14['SeriesCode']=='EN_MAR_CHLDEV'].pivot_table(columns='TimePeriod', index='GeoAreaName', values='Value', aggfunc='mean')
# minMax transform 
sdg14EZZ[[2012,2014,2021]]

TimePeriod,2012,2014,2021
GeoAreaName,,,
Belgium,1.58103,1.18214,0.39370
Bulgaria,0.48363,1.86093,6.87510
Croatia,0.38562,0.78792,0.46349
Cyprus,0.07214,0.03834,1.48557
Denmark,3.92073,2.79938,2.58100
Estonia,3.81015,1.24961,0.90704
Finland,6.19252,2.44967,2.04491
France,2.42255,6.42791,3.41925
Germany,2.40504,1.62128,2.37366


### 14.1

#### (b) Floating Plastic Debris Density


In [390]:
# Plastic Waste kg/ha, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ENV_WASGEN/

wasteG = pd.read_csv(os.path.join(rawPath,'env_wasgen__custom_5593208_page_linear.csv')) #read file
wasteG = wasteG[wasteG['geo'].isin(countryAbb)] #filter countries
wasteG['geo'] = wasteG['geo'].map(abbrev_to_country).fillna(wasteG['geo']) #change abb to name
wasteG  = wasteG[wasteG['unit']=='KG_HAB'].pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean') #pivot table 

mr = wasteG.columns[-1] # most recent year
wasteG
#maxMin transformation as (x-max)/(max-min) * 100 --> converts to 0-100 scale with 0=worst, 100=best
(wasteG[[2012, 2014, mr]].max().max() - wasteG[[2012,2014, mr]])/(wasteG[[2012,2014, mr]].max().max() - wasteG[[2012,2014, mr]].min().min())*100 

TIME_PERIOD,2012,2014,2020
geo,,,
Belgium,54.621849,0.000000,31.932773
Bulgaria,92.436975,81.512605,59.663866
Croatia,96.638655,94.117647,78.151261
Cyprus,100.000000,99.159664,88.235294
Denmark,88.235294,89.075630,84.033613
Estonia,89.915966,83.193277,77.310924
Finland,89.915966,86.554622,84.033613
France,83.193277,82.352941,73.949580
Germany,78.151261,76.470588,73.109244


In [398]:
# Recovery rates for packaging waste, Plastic Packaging, Percentage, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ten00062/

wasteR = pd.read_csv(os.path.join(rawPath,'ten00062__custom_5244970_page_linear.csv'))
wasteR = wasteR[wasteR['geo'].isin(countryAbb)]
wasteR['geo'] = wasteR['geo'].map(abbrev_to_country).fillna(wasteR['geo'])
wasteR = wasteR.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')

mr = wasteR.columns[-1] # most recent year
wasteR = wasteR.ffill(axis = 1) #fill empty values with last available year
wasteR[[2012, 2014, mr]]

TIME_PERIOD,2012,2014,2020
geo,,,
Belgium,92.7,98.3,99.4
Bulgaria,44.1,64.1,50.6
Croatia,45.4,38.0,34.1
Cyprus,44.8,46.6,51.9
Denmark,99.4,98.2,73.1
Estonia,44.0,75.9,87.4
Finland,51.0,68.3,99.4
France,64.0,64.0,71.8
Germany,99.7,99.8,99.9


#### (a) Index of coastal eutrophication


In [299]:
#Gross nutrient balance (Nitrogen kg/ha), Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/AEI_PR_GNB__custom_153613/

nitro = pd.read_csv(os.path.join(rawPath,'aei_pr_gnb__custom_153613_page_linear.csv'))
nitro = nitro[nitro['geo'].isin(countryAbb)]
nitro['geo'] = nitro['geo'].map(abbrev_to_country).fillna(nitro['geo'])
nitro.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')

TIME_PERIOD,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
geo,,,,,,,,,,,,,,,,,,,,,
Belgium,254.0,241.0,232.0,233.0,225.0,189.9,180.0,169.5,151.6,145.6,...,129.3,142.0,143.2,143.1,137.9,131.7,132.0,NaN,NaN,NaN
Bulgaria,29.0,42.0,33.0,21.0,30.0,28.2,26.8,21.6,31.3,18.9,...,20.3,14.2,11.9,24.2,15.8,27.6,28.0,71.10,65.80,61.8
Croatia,NaN,NaN,NaN,NaN,NaN,122.8,122.8,109.3,119.4,111.1,...,61.6,80.8,93.2,86.3,43.9,48.3,59.7,35.40,65.10,57.6
Cyprus,207.0,211.0,216.0,214.0,197.0,186.0,191.2,209.0,181.1,172.3,...,177.6,191.4,198.7,184.4,178.5,194.3,194.0,NaN,NaN,NaN
Denmark,156.0,139.0,132.0,132.0,125.0,131.7,127.4,124.4,117.5,121.3,...,86.5,90.0,88.0,83.4,87.1,79.8,80.0,NaN,NaN,NaN
Estonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.5,...,25.1,30.8,32.4,28.0,NaN,NaN,NaN,NaN,NaN,NaN
Finland,79.0,71.0,66.0,74.0,64.0,54.9,59.1,54.8,55.8,53.1,...,38.0,57.1,50.2,47.5,46.5,47.5,49.5,47.40,51.50,58.0
France,55.0,58.0,53.0,51.0,55.0,42.6,49.5,39.6,61.0,35.6,...,29.8,36.3,49.1,23.5,29.7,29.6,37.3,47.50,33.70,40.8
Germany,109.0,106.0,99.0,97.0,102.0,110.0,97.5,100.7,113.7,84.5,...,65.4,77.7,88.6,75.1,78.9,65.9,81.8,69.05,61.51,77.9


### 14.3

#### Floating Plastic Debris Density

In [ ]:
#Gross nutrient balance (Nitrogen kg/ha), Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/AEI_PR_GNB__custom_153613/

nitro = pd.read_csv(os.path.join(rawPath,'aei_pr_gnb__custom_153613_page_linear.csv'))
nitro = nitro[nitro['geo'].isin(countryAbb)]
nitro['geo'] = nitro['geo'].map(abbrev_to_country).fillna(nitro['geo'])
nitro.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')